In [5]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

%run dots_model.ipynb
%run TFSessionManager.ipynb


class GameResult:
    def __init__(self, winner):
        self.winner = winnner
        
def face_off(p1, p2, games_per_match, matches):
    matches_for_1 = 0
    for _ in range(matches):
        wins_for_1 = 0
        for _ in range(games_per_match):
            g = Game(p1, p2)
            if (g.play_game() == p1.ind):
                wins_for_1 += 1
        if (wins_for_1 > games_per_match - wins_for_1):
            matches_for_1 += 1
    print("{} won {} matches, {} won {}".format(p1.name, matches_for_1, p2.name, matches - matches_for_1))
    return matches_for_1, matches - matches_for_1, 0

def battle(p1, p2, num_games, silent=False):
    return face_off(p1, p2, 1, num_games)
    
if True:
    board = Board()
    nnplayer = NNQPlayer("QLearner2", WHITE)
    randomplayer = RandomPlayer("Random", BLACK)
    greedyplayer = GreedyPlayer("Greedy", WHITE)
    #tq_player = TQPlayer()

    p1_wins = []
    p2_wins = []
    draws = []
    game_number = []
    players = []
    game_counter = 0

    num_battles = 1
    games_per_battle = 1
    num_training_battles = 1

    TFSessionManager.set_session(tf.Session())

    TFSessionManager.get_session().run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('log', TFSessionManager.get_session().graph)

    # nnplayer rndplayer mm_player
    p1_t = greedyplayer
    p2_t = randomplayer

    p1 = p1_t
    p2 = p2_t

    p1.ind = 1
    p2.ind = 2

    # nnplayer.training= False
    # nnplayer2.training= False

    for i in range(num_training_battles):
        p1win, p2win, draw = battle(p1_t, p2_t, games_per_battle, False)
        p1_wins.append(p1win)
        p2_wins.append(p2win)
        draws.append(draw)
        game_counter = game_counter + 1
        game_number.append(game_counter)
        if (i % 100 == 0):
            print("games=" + str(i))
    # nnplayer.training= False
    # nnplayer2.training= False

    p2_t = randomplayer
    p1_t = nnplayer

    p1 = p1_t
    p2 = p2_t

    p1.ind = 1
    p2.ind = 2

    p1_wins = []
    p2_wins = []
    draws = []
    game_number = []
    game_counter = 0

    for i in range(num_battles):
        p1win, p2win, draw = battle(p1, p2, games_per_battle, False)
        p1_wins.append(p1win)
        p2_wins.append(p2win)
        draws.append(draw)
        game_counter = game_counter + 1
        game_number.append(game_counter)

    writer.close()
    TFSessionManager.set_session(None)

    tf.summary.scalar('loss', nnplayer.nn.q_values)

    p = plt.plot(game_number, p1_wins, 'g-', game_number, p2_wins, 'b-')

    plt.show()



Greedy won 10 matches, Random won 0


NameError: name 'NNQPlayer' is not defined

In [2]:
def create_temp_model():
    with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
        x = tf.placeholder(tf.float32, shape=[None, BOARD_EDGES], name='X')
        y = tf.placeholder(tf.float32, shape=[None, BOARD_EDGES], name='Y')

        net = x
        net = tf.layers.dense(net, BOARD_EDGES*8*BOARD_EDGES, tf.nn.relu,
                                       kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                                       name='hidden')
        output_logits = tf.layers.dense(net, BOARD_EDGES, None,
                                       kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
                                       name='q_valuesz')
        predictions = tf.nn.softmax(output_logits, name='predictions')


        # Network predictions
        cls_prediction = tf.argmax(output_logits, axis=1, name='prediction')

        # Define the loss function, optimizer, and accuracy
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output_logits), name='loss')
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001, name='Adam-op').minimize(loss)
        correct_prediction = tf.equal(tf.argmax(output_logits, 1), tf.argmax(y, 1), name='correct_pred')
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

        saver = tf.train.Saver()
    
def get_next_batch(x_set, y_set, start, end):
    return x_set[start : end], y_set[start : end]

def build_and_save_model():
    epochs = 10            # Total number of training epochs
    batch_size = 100        # Training batch size
    display_freq = 100      # Frequency of displaying the training results

    # Create the op for initializing all variables
    init = tf.global_variables_initializer()

    accs = []
    iters = []

    create_temp_model()
    
    starting_ind = 0

    # Launch the graph (session)
    #with tf.Session() as sess:
    sess = tf.Session()
    sess.run(init)
    writer = tf.summary.FileWriter('log', TFSessionManager.get_session().graph)
    global_step = 0
    # Number of training iterations in each epoch
    num_tr_iter = 100 #int(len(y_train) / batch_size)
    for epoch in range(epochs):
        print('Training epoch: {}'.format(epoch + 1))

        #x_train, y_train = randomize(x_train, y_train)
        for iteration in range(num_tr_iter):
            global_step += 1
            start = iteration * batch_size + starting_ind
            end = (iteration + 1) * batch_size + starting_ind
            #start = iteration * batch_size
            #end = (iteration + 1) * batch_size
            #x_batch, y_batch = get_next_batch(x_train, y_train, start, end)
            x_batch, y_batch = generateData(batch_size)
            # Run optimization op (backprop)
            feed_dict_batch = {x: x_batch, y: y_batch}
            sess.run(optimizer, feed_dict=feed_dict_batch)

            if iteration % display_freq == 0:
                # Calculate and display the batch loss and accuracy
                loss_batch, acc_batch = sess.run([loss, accuracy],
                                                 feed_dict=feed_dict_batch)

                accs.append(acc_batch)
                iters.append(global_step)

                print("iter {0:3d}:\t Loss={1:.2f},\tTraining Accuracy={2:.01%}".
                      format(iteration, loss_batch, acc_batch))

    p = plt.plot(iters, accs, 'b-')
    plt.show()

    #print("saving model")
    #save_path = saver.save(sess, "/tmp/dots_model", global_step=epoch)
    #print("Model saved in path: %s" % save_path)

    print('done')